In [ ]:
%pip install mediapipe opencv-python pyngrok
%pip install pyautogui 

In [ ]:
import mediapipe as mp
import cv2
import pyautogui  # for mouse control

# initialize
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# camera
cap = cv2.VideoCapture(0)

numeric_value = 50 

click_flag = False
mouse_position = (0, 0)  

# buttons' positions initialization
buttons = [
    {"label": "Button 1", "x1": 50, "y1": 50, "x2": 200, "y2": 100, "color": (200, 200, 200), "hover": False, "clicked": False},
    {"label": "Button 2", "x1": 50, "y1": 150, "x2": 200, "y2": 200, "color": (200, 200, 200), "hover": False, "clicked": False},
]

# mouse click func
def mouse_callback(event, x, y, flags, param):
    global mouse_position, buttons
    mouse_position = (x, y)

    if event == cv2.EVENT_LBUTTONDOWN:
        for button in buttons:
            if button["x1"] <= x <= button["x2"] and button["y1"] <= y <= button["y2"]:
                button["clicked"] = True
                print(f"{button['label']} clicked")


cv2.namedWindow("Hand Gesture Recognition with Interface")
cv2.setMouseCallback("Hand Gesture Recognition with Interface", mouse_callback)

# mediapipe hand detection module initialization
with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Kamera girişi okunamadı!")
            break
        
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        results = hands.process(rgb_frame)
        
        screen_width, screen_height = pyautogui.size()
        
        # button drawing
        for button in buttons:
            color = button["color"]
            if button["clicked"]:
                color = (0, 255, 0)  # make the color green after click
            elif button["hover"]:
                color = (180, 180, 250)  # hovering color
            cv2.rectangle(frame, (button["x1"], button["y1"]), (button["x2"], button["y2"]), color, -1)
            cv2.putText(frame, button["label"], (button["x1"] + 10, button["y1"] + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        cv2.putText(frame, f"Value: {numeric_value}", (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (139, 0, 139), 2)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
                
                cursor_x = int(index_tip.x * frame.shape[1])
                cursor_y = int(index_tip.y * frame.shape[0])
                
                pyautogui.moveTo(cursor_x * screen_width // frame.shape[1], cursor_y * screen_height // frame.shape[0])
                cv2.putText(frame, "Cursor Moving", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                # hover state over buttons
                for button in buttons:
                    if button["x1"] <= mouse_position[0] <= button["x2"] and button["y1"] <= mouse_position[1] <= button["y2"]:
                        button["hover"] = True
                    else:
                        button["hover"] = False
                
                # V gesture for clicking
                if (index_tip.y < middle_tip.y and ring_tip.y > middle_tip.y and pinky_tip.y > middle_tip.y):
                    if not click_flag:
                        click_flag = True
                        cv2.putText(frame, "Click Detected", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2) 
                        
                       
                        for button in buttons:
                            if button["hover"]:
                                button["clicked"] = True
                                print(f"{button['label']} clicked")
                else:
                    click_flag = False

                # Scroll Down 
                thumb_down = thumb_tip.y > thumb_ip.y + 0.05 
                if thumb_down and abs(thumb_tip.x - thumb_ip.x) < 0.1:
                    numeric_value = max(0, numeric_value - 1)
                    cv2.putText(frame, "Scroll Down", (300, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
                # Scroll Up
                thumb_up = (
                    thumb_tip.y < thumb_ip.y - 0.05 and all(finger.y > thumb_tip.y + 0.05 for finger in [index_tip, middle_tip, ring_tip, pinky_tip])
                )
                if thumb_up and abs(thumb_tip.x - thumb_ip.x) < 0.1:
                    numeric_value = min(100, numeric_value + 1)
                    cv2.putText(frame, "Scroll Up", (300, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Hand Gesture Recognition with Interface", frame)
        
        # 'q' for exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# close camera
cap.release()
cv2.destroyAllWindows()